Tool updated: 2024-06-03, Henrik Loecke

Tool summary:
This tool creates RAWN sheets, based on the MIKE+ model and MPF population file. It traces through the model 


In [220]:
#Permanent cell 1
import arcpy
import pandas as pd
import sqlite3
import math
import numpy as np
import os
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import ctypes
import traceback
import shutil
MessageBox = ctypes.windll.user32.MessageBoxA

In [189]:
#Permanent cell 2
def sql_to_df(sql,model):
    con = sqlite3.connect(model)
    df = pd.read_sql(sql, con)
    con.close()
    return df

def execute_sql(sqls,model):
    con = sqlite3.connect(model)
    cur = con.cursor()
    if type(sqls) == list:
        for sql in sqls:
            cur.execute(sql)
    else:         
        cur.execute(sqls)
    cur.close()
    con.commit()
    con.close()

In [190]:
#Permanent cell 3
# User Input, to move to separate sheet so no permanent cell
#stop trace if more pipes than max_steps traced from catchment, must be an endless loop. 

model = 'NSSA'

max_steps = 1000 

check_scenario = True #Scenario check is for max active altid. In some hypothetical setups it can be incorrect. 
#                      If you verify the scenario is correct (open model to check) but get the error, set this check to False
#                      A more robust check may be developed in the future.
update_field_in_model = True
update_field = 'Description'
global_output_folder = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\Rawn_Tool\Output"

if model == 'NSSA':   
    model_output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\04_ANALYSIS_WORK\RAWN_From_Model"
    model_path = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\NSSA_Base_2018pop.sqlite"
    pop_book = r"\\prdsynfile01\LWS_Modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\MPF4_Temp_Hold\NSSA_Master_Population_File_4_No_2237_ResArea.xlsx"
    pop_sheet = 'MPF Update 4'
    scenario = 'Base'
    
if model == 'FSA':   
    model_output_folder = r'\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\RAWN_From_Model'
    model_path = r''
    pop_book = r"\\prdsynfile01\LWS_Modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\MPF4_Temp_Hold\NSSA_Master_Population_File_4_No_2237_ResArea.xlsx"
    pop_sheet = ''
    scenario = '2030_Network'


sewer_area = model
gdb_name = 'RAWN.gdb'
gdb_name_dissolve = 'RAWN_Dissolve.gdb' #To keep clutter out of main database

#Options to skip time consuming steps during debug, must be True during production runs
run_dissolve = True
run_jpg = True
run_import = True
run_html = False


In [191]:
#Permanent cell 4
#Set up column names

try:

    years = [2060,2070,2080,2090,2100]
    categories = ['res','com','ind','inst','infl','infi']

    mpf_col_dict = {}

    area_col_dict = {}
    area_col_dict['res'] = 'Area_Res'
    area_col_dict['com'] = 'Area_Com'
    area_col_dict['ind'] = 'Area_Ind'
    area_col_dict['inst'] = 'Area_Inst'
    area_col_dict['ini'] = 'Area_Total'

    per_unit_dict = {}
    per_unit_dict['res'] = 320
    per_unit_dict['com'] = 33700 
    per_unit_dict['ind'] = 56200
    per_unit_dict['inst'] = 33700
    per_unit_dict['infl'] = 5600
    per_unit_dict['infi'] = 5600

    unit_dict = {}
    unit_dict['res'] = 'L/c/d'
    unit_dict['com'] = 'L/ha/d'
    unit_dict['ind'] = 'L/ha/d'
    unit_dict['inst'] = 'L/ha/d'
    unit_dict['infl'] = 'L/ha/d'


    header_dict = {}
    # header_dict['gen'] = ['GENERAL INFO',['TYPE','MODELID','CATCHMENT','ID','YEAR','LOCATION']]
    header_dict['gen'] = ['GENERAL INFO',['TYPE','CATCHMENT','YEAR','LOCATION']]
    header_dict['res'] = ['RESIDENTIAL',['AREA (Ha)','POPULATION','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['com'] = ['COMMERCIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['ind'] = ['INDUSTRIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['inst'] = ['INSTITUTIONAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['ini'] = ['INFLOW / INFILTRATION',['AREA (Ha)','INFLOW (L/s)','INFILTRATION (L/s)']]
    header_dict['flow'] = ['FLOWS',['AVG. SAN. FLOW (L/s)','ADWF (L/s)','PWWF (L/s)']]

    avg_calc_dict = {}
    avg_calc_dict['res'] = ['RESIDENTIAL','POPULATION','AVG. FLOW (L/s)']
    avg_calc_dict['com'] = ['COMMERCIAL','AREA (Ha)','AVG. FLOW (L/s)']
    avg_calc_dict['ind'] = ['INDUSTRIAL','AREA (Ha)','AVG. FLOW (L/s)']
    avg_calc_dict['inst'] = ['INSTITUTIONAL','AREA (Ha)','AVG. FLOW (L/s)']
    avg_calc_dict['infl'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFLOW (L/s)']
    avg_calc_dict['infi'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFILTRATION (L/s)']

    header_tuples = []
    for header in header_dict:
        for sub_header in (header_dict[header][1]):
            header_tuples.append((header_dict[header][0],sub_header))
    header_tuples

    # columns_multiindex = pd.MultiIndex.from_tuples(header_tuples,names=['Category', 'Subcategory'])
    columns_multiindex = pd.MultiIndex.from_tuples(header_tuples)
    df_template = pd.DataFrame(columns=columns_multiindex)

    info_list = []
    for item in unit_dict:
        info_list.append([avg_calc_dict[item][0],per_unit_dict[item],unit_dict[item]])
    info_df = pd.DataFrame(info_list,columns=['DESCRIPTION','AVG. FLOW','UNITS'])
    info_df.set_index('DESCRIPTION',inplace=True)
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 4', b'Error', 0)
    raise ValueError("Error")

In [192]:
#Permanent cell 5
#Import population
try:
    pop_df = pd.read_excel(pop_book,sheet_name=pop_sheet,dtype={'Catchment': str})#[['Catchment','Year','Pop_Total']]
    pop_df.rename(columns={"Pop_Total": "Population"},inplace=True)
    pop_df = pop_df[['Catchment','Year','Pop_ResLD','Pop_ResHD','Pop_Mixed','Population','Area_ResLD','Area_ResHD','Area_Mixed','Area_Com','Area_Ind','Area_Inst']]
    pop_df['Area_Res'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed
    pop_df['Area_Total'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed + pop_df.Area_Com + pop_df.Area_Ind + pop_df.Area_Inst
    pop_df['Population_Sum_Check'] = pop_df.Pop_ResLD + pop_df.Pop_ResHD + pop_df.Pop_Mixed
    pop_sum_total_col = int(pop_df.Population.sum())
    pop_sum_sub_cols = int(pop_df.Pop_ResLD.sum() + pop_df.Pop_ResHD.sum() + pop_df.Pop_Mixed.sum())
    pop_df['Key'] = sewer_area + '@' + pop_df.Catchment + '@' + pop_df['Year'].astype(str)
    pop_df.set_index('Key',inplace=True)

    if pop_sum_total_col != pop_sum_sub_cols:
          raise ValueError("Error. The sum of 'Population' (" + str(pop_sum_total_col) + ") is different than the sum of 'Pop_ResLD' + 'Pop_ResHD' + 'Pop_Mixed' (" + str(pop_sum_sub_cols) + ")") 

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 5', b'Error', 0)
    raise ValueError("Error")

In [193]:
#Permanent cell 6
#Import model data

try:
    node_types = {}
    node_types[1] = 'Manhole'
    node_types[2] = 'Basin'
    node_types[3] = 'Outlet'
    node_types[4] = 'Junction'
    node_types[5] = 'Soakaway'
    node_types[6] = 'River Junction'

    sql = "SELECT max(AltID) FROM msm_Loadpoint WHERE Active = 1"
    altid = sql_to_df(sql,model_path).iloc[0,0]

    if altid = 0:
        active_scenario = 'Base'
    else:
        sql = "SELECT MUID FROM m_ScenarioManagementAlternative WHERE GroupID = 'CS_Network' AND AltID = " + altid
        active_scenario = sql_to_df(sql,model_path).iloc[0,0]

    if scenario != active_scenario:
        raise ValueError(f'Scenario {scenario} was requested in the user input but scenario {active_scenario} is active in the model')

    sql = "SELECT catchid AS Catchment, nodeid AS Connected_Node FROM msm_Catchcon WHERE Active = 1"
    catchments = sql_to_df(sql,model_path)

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], uplevel AS Outlet_Level FROM msm_Link WHERE Active = 1"
    lines = sql_to_df(sql,model_path)

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Orifice WHERE Active = 1"
    orifices = sql_to_df(sql,model_path)
    lines = pd.concat([lines,orifices])

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Valve WHERE Active = 1"
    valves = sql_to_df(sql,model_path)
    lines = pd.concat([lines,valves])

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], crestlevel AS Outlet_Level FROM msm_Weir WHERE Active = 1"
    weirs = sql_to_df(sql,model_path)
    lines = pd.concat([lines,weirs])

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], startlevel AS Outlet_Level FROM msm_Pump WHERE Active = 1"
    pumps = sql_to_df(sql,model_path)
    lines = pd.concat([lines,pumps])

    lines['Outlet_Level'].fillna(-9999, inplace=True)

    sql = "SELECT muid, acronym, assetname FROM msm_Node WHERE active = 1"
    node_id_df = sql_to_df(sql,model_path)
    node_id_df = node_id_df[(node_id_df.assetname.str[:2]=='MH') & (node_id_df.assetname.str.len() > 2) & (node_id_df.acronym.notna())]
    node_id_df.rename(columns={'muid':'Node'},inplace=True)
    node_id_df['ID'] = node_id_df.acronym + '_' + node_id_df.assetname
    node_id_df = node_id_df[['Node','ID']]
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 6', b'Error', 0)
    raise ValueError("Error")

In [194]:
#Permanent cell 7
#Trace the model

try:
    accumulated_catchment_set = set()
    accumulated_node_set = set()

    for index1, row1 in catchments.iterrows():
        catchment = row1['Catchment']
        nodes = [row1['Connected_Node']]
        start_node = row1['Connected_Node']
        steps = 0

        accumulated_catchment_set.add((start_node,catchment))

        while steps <= max_steps:
            steps += 1
            downstream_df = lines[lines['From'].isin(nodes)]

            if len(downstream_df) > 0:
                nodes = list(downstream_df.To.unique())

                nodes = [node for node in nodes if len(node)>0]
                for node in nodes:
                    accumulated_catchment_set.add((node,catchment))       
            else:
                break
            if steps == max_steps:
                raise ValueError("Maximum steps were reached, indicating a loop. Start catchment is '" + catchment + "'")

            accumulated_catchment_set.add((node,catchment))

    accumulation_df = pd.DataFrame(accumulated_catchment_set,columns=['Node','Catchment'])
    accumulation_df = pd.merge(accumulation_df,node_id_df,how='inner',on=['Node'])
    data = {
        ('GENERAL INFO', 'CATCHMENT'): accumulation_df.Catchment,
        ('GENERAL INFO', 'NODE'): accumulation_df.Node,
        ('GENERAL INFO', 'ID'): accumulation_df.ID,
    }

    # Create a DataFrame with MultiIndex columns
    accumulation_df = pd.DataFrame(data)
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 7', b'Error', 0)
    raise ValueError("Error")



In [195]:
#Permanent cell 8
#Calculate RAWN

try:
    catchments = list(pop_df.Catchment.unique())

    catchment_df = df_template.copy()
    for catchment in catchments:
        for year in years:
            key = model + '@' + catchment + '@' + str(year)
            catchment_df.loc[key,('GENERAL INFO','TYPE')] = 'Manhole'
            catchment_df.loc[key,('GENERAL INFO','CATCHMENT')] = catchment
            catchment_df.loc[key,('GENERAL INFO','YEAR')] = year
            catchment_df.loc[key,('GENERAL INFO','LOCATION')] = model
            for area_col_dict_key in area_col_dict:
                catchment_df.loc[key,(header_dict[area_col_dict_key][0],'AREA (Ha)')] = pop_df.loc[key,area_col_dict[area_col_dict_key]]
            catchment_df.loc[key,('RESIDENTIAL','POPULATION')] = pop_df.loc[key,'Population']
            san_flow = 0
            adwf = 0
            for avg_calc_dict_key in avg_calc_dict:
                input1 = catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][1])]
                input2 = per_unit_dict[avg_calc_dict_key]
                avg_flow = input1 * input2 / 86400
                adwf += avg_flow
                if avg_calc_dict_key not in ['infl','infi']:
                    san_flow += avg_flow
                catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][2])] = avg_flow
            catchment_df.loc[key,('FLOWS','AVG. SAN. FLOW (L/s)')] = san_flow
            catchment_df.loc[key,('FLOWS','ADWF (L/s)')] = adwf


    catchment_node_df = accumulation_df.merge(catchment_df,on=[('GENERAL INFO','CATCHMENT')],how='inner')
    node_df = catchment_node_df.copy()
    node_df.drop(columns=[('GENERAL INFO','CATCHMENT')],inplace=True)
    node_df = node_df.groupby([('GENERAL INFO','NODE'),('GENERAL INFO','TYPE'),('GENERAL INFO','YEAR'),('GENERAL INFO','LOCATION'),('GENERAL INFO','ID')]).sum()
    node_df.reset_index(inplace=True)
    node_df[('RESIDENTIAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (node_df[('RESIDENTIAL','POPULATION')] / 1000) ** 0.5)) * node_df[('RESIDENTIAL','AVG. FLOW (L/s)')]
    node_df[('COMMERCIAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (per_unit_dict['com'] * node_df[('COMMERCIAL','AREA (Ha)')]/(per_unit_dict['res'] * 1000)) ** 0.5))*node_df[('COMMERCIAL','AVG. FLOW (L/s)')]*0.8
    node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (per_unit_dict['inst'] * node_df[('INSTITUTIONAL','AREA (Ha)')] / (per_unit_dict['res'] * 1000)) ** 0.5)) * node_df[('INSTITUTIONAL','AVG. FLOW (L/s)')]

    mask = node_df[('INDUSTRIAL', 'AREA (Ha)')] != 0 #Avoid error from log(0)
    node_df.loc[mask, ('INDUSTRIAL', 'PEAK FLOW (L/s)')] = (
        0.8 * (1 + 14 / (4 + (node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] * per_unit_dict['ind'] / (per_unit_dict['res'] * 1000)) ** 0.5)) *
        np.where(
            node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] < 121,
            1.7,
            2.505 - 0.1673 * np.log(node_df[('INDUSTRIAL', 'AREA (Ha)')][mask])
        ) * node_df[('INDUSTRIAL', 'AVG. FLOW (L/s)')][mask]
    )

    node_df[('FLOWS','PWWF (L/s)')] = (
        node_df[('RESIDENTIAL','PEAK FLOW (L/s)')] +
        node_df[('COMMERCIAL','PEAK FLOW (L/s)')] +
        node_df[('INDUSTRIAL','PEAK FLOW (L/s)')] +
        node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] +
        node_df[('INFLOW / INFILTRATION','INFLOW (L/s)')] +
        node_df[('INFLOW / INFILTRATION','INFILTRATION (L/s)')]
    )
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 8', b'Error', 0)
    raise ValueError("Error")



In [196]:
#Permanent cell 9
#Import GIS from the model

try:
    if run_import:

        out_path = global_output_folder + '\\' + gdb_name

        if not os.path.isdir(out_path):
            arcpy.management.CreateFileGDB(global_output_folder, gdb_name)

        arcpy.env.workspace = out_path
        sr = arcpy.SpatialReference(26910)

        layers = ['msm_CatchCon','msm_Catchment','msm_Link','msm_Node','msm_Pump','msm_Weir','msm_Orifice','msm_Valve']

        for layer in layers:
            print(layer)
        #     arcpy.conversion.FeatureClassToFeatureClass(model_path + '\\' + layer, out_path, layer)

            arcpy.management.MakeFeatureLayer(model_path + '\\' + layer, "temp_layer", "Active = 1")

            if arcpy.Exists(layer):
                arcpy.management.DeleteFeatures(layer)
                arcpy.management.Append("temp_layer", layer, "NO_TEST")
            else:    
                arcpy.conversion.FeatureClassToFeatureClass("temp_layer", out_path, layer)
                if layer == 'msm_Catchment':
                    arcpy.management.AddField('msm_catchment', "Drains_To", "TEXT")

            arcpy.management.Delete("temp_layer")
            arcpy.DefineProjection_management(layer, sr)
            
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 9', b'Error', 0)
    raise ValueError("Error")


msm_CatchCon
msm_Catchment
msm_Link
msm_Node
msm_Pump
msm_Weir
msm_Orifice
msm_Valve


In [197]:
#Permanent cell 10
#Create merge_df,minimizing the number of computation heavy dissolves that need to be done.

try:
    merge_set = set()

    rank_df = accumulation_df[[('GENERAL INFO','NODE'),('GENERAL INFO','CATCHMENT')]].groupby([('GENERAL INFO','NODE')]).count()

    # rank_df.reset_index(inplace=True)
    rank_df.columns = ['Catchment_Count']
    max_catchments = max(rank_df.Catchment_Count)
    rank_df.sort_values(by=['Catchment_Count'],inplace=True)
    # rank_df.reset_index(inplace=True)

    catchment_list = []
    merge_set = set()
    for index, row in rank_df.iterrows():

        catchments = list(accumulation_df[accumulation_df[('GENERAL INFO','NODE')]==index][('GENERAL INFO','CATCHMENT')].unique())
        catchments = tuple(sorted(catchments))
    #     rank_df.loc[index,'Catchments'] = catchments
        catchment_list.append(catchments)
        merge_set.add(catchments)


    rank_df['Catchments'] = catchment_list
    rank_df['Node'] = rank_df.index
    print(len(merge_set))

    merge_list = []
    for i, catchments in enumerate(merge_set):
        merge_id = 'Merge_ID_' + str(i)
        merge_list.append([merge_id,catchments])

    merge_df = pd.DataFrame(merge_list,columns=['Merge_ID','Catchments'])
    merge_df['Catchment_Count'] = merge_df['Catchments'].apply(len)
    merge_df.sort_values(by=['Catchment_Count'],ascending=False,inplace=True)
    merge_df.reset_index(inplace=True,drop=True)

    simpler_merge = []
    for index1, row1 in merge_df.iterrows():
        catchments1 = list(row1['Catchments'])
        for index2, row2 in merge_df[index1+1:].iterrows():
            catchments2 = row2['Catchments']

            if len(catchments1) >= len(catchments2):
                if all(item in catchments1 for item in catchments2):
                    catchments1 = [catchment for catchment in catchments1 if catchment not in catchments2]
                    catchments1.append(row2['Merge_ID'])
        simpler_merge.append(catchments1)

    merge_df['To_Dissolve'] = simpler_merge
    merge_df.sort_values(by=['Catchment_Count'],inplace=True)
    merge_df.reset_index(inplace=True,drop=True)

    rank_df = pd.merge(rank_df,merge_df[['Merge_ID','Catchments']], on=['Catchments'],how='inner')
    rank_df.set_index('Node',inplace=True)
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 10', b'Error', 0)
    raise ValueError("Error")



103


In [203]:
rank_df.loc['6786','Merge_ID']

'Merge_ID_98'

In [204]:
merge_df

,Merge_ID,Catchments,Catchment_Count,To_Dissolve
0,Merge_ID_70,"(2169,)",1,[2169]
1,Merge_ID_98,"(2122,)",1,[2122]
2,Merge_ID_38,"(2052,)",1,[2052]
3,Merge_ID_67,"(2119, 2122)",2,"[2119, Merge_ID_98]"
4,Merge_ID_35,"(2128, 2136)",2,"[2128, 2136]"
...,...,...,...,...
98,Merge_ID_32,"(2059, 2062, 2063, 2064, 2066, 2068, 2069, 207...",95,"[Merge_ID_7, Merge_ID_53]"
99,Merge_ID_3,"(2059, 2061, 2062, 2063, 2064, 2065, 2066, 206...",98,"[2061, 2065, 2067, Merge_ID_32]"
100,Merge_ID_29,"(2059, 2060, 2061, 2062, 2063, 2064, 2065, 206...",99,"[2060, Merge_ID_3]"
101,Merge_ID_87,"(10108, 10110, 2059, 2060, 2061, 2062, 2063, 2...",104,"[10108, 10110, 2070, 2075, 2089, Merge_ID_29]"


In [226]:
#Permanent cell 11
#Run dissolve

gdb_name_dissolve = 'teeeest.gdb'

try:
    if run_dissolve:
        out_path = global_output_folder + '\\' + gdb_name
        arcpy.env.workspace = out_path  
        arcpy.env.addOutputsToMap = False
        if run_dissolve:
            if arcpy.Exists(gdb_name_dissolve):
                arcpy.management.Delete(gdb_path)
            arcpy.management.CreateFileGDB(global_output_folder, gdb_name_dissolve)
            dissolve_path = global_output_folder + '\\' + gdb_name_dissolve
            arcpy.conversion.FeatureClassToFeatureClass('msm_Catchment', dissolve_path, 'Node_Catchment')
        #     arcpy.env.workspace = dissolve_path
            arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Drains_To", "TEXT")
            arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Merge_ID", "TEXT")
            arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Merge_ID_Temp", "TEXT")
            arcpy.management.CalculateField(dissolve_path + '\\Node_Catchment', "Merge_ID", "!muid!", "PYTHON3")
            for index, row in merge_df[:6].iterrows():
                arcpy.management.CalculateField(dissolve_path + '\\Node_Catchment', "Merge_ID_Temp", "''", "PYTHON3")
                nodes = list(rank_df[rank_df.Merge_ID==row["Merge_ID"]].index)
                print(f'Dissolving for {row["Merge_ID"]}, {index} of {max(merge_df.index)} at time {datetime.datetime.now()}')
                if row['Catchment_Count'] == 1:
                    arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
                    where_clause = f"muid = '{row['To_Dissolve'][0]}'"
                    arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
                    arcpy.management.CalculateField("temp_layer", "Merge_ID", f"'{row['Merge_ID']}'", "PYTHON3")
                    arcpy.conversion.FeatureClassToFeatureClass('temp_layer', dissolve_path, 'Dissolve_Temp')
                    arcpy.management.Delete("temp_layer")
                    
                else:
                    arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
                    catchments = row['To_Dissolve']
                    catchments_sql = ', '.join([f"'{muid}'" for muid in catchments])
                    where_clause = f"Merge_ID in ({catchments_sql})"
                    arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
                    arcpy.management.CalculateField("temp_layer", "Merge_ID_Temp", f"'{row['Merge_ID']}'", "PYTHON3")
                    arcpy.management.Dissolve("temp_layer",dissolve_path + '\\Dissolve_Temp', "Merge_ID_Temp", "", "MULTI_PART")
                    arcpy.management.Delete("temp_layer")
                    arcpy.management.CalculateField(dissolve_path + '\\Dissolve_Temp', "Merge_ID", f"'{row['Merge_ID']}'", "PYTHON3")

                for node in nodes:
                    arcpy.management.CalculateField(dissolve_path + '\\Dissolve_Temp', "Drains_To", f"'{node}'", "PYTHON3")
                    arcpy.management.Append(dissolve_path + '\\Dissolve_Temp', dissolve_path + '\\Node_Catchment', "NO_TEST")

                arcpy.management.Delete(dissolve_path + '\\Dissolve_Temp')



        #Delete the features without a Drains_To
        arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
        where_clause = f"Drains_To IS NULL"
        arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
        arcpy.management.DeleteFeatures("temp_layer")  
        arcpy.management.Delete("temp_layer")

        #Append the features into the official Node_Catchment layer
        arcpy.management.MakeFeatureLayer("Node_Catchment", "Temp_Layer")
        arcpy.management.DeleteFeatures("Temp_Layer")
        arcpy.management.Append(dissolve_path + '\\Node_Catchment', "Temp_Layer", "NO_TEST")
        arcpy.management.Delete("temp_layer")
        
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 11', b'Error', 0)
    raise ValueError("Error")




Dissolving for Merge_ID_70, 0 of 102 at time 2024-06-03 15:20:24.531315
Dissolving for Merge_ID_98, 1 of 102 at time 2024-06-03 15:20:47.293639
Dissolving for Merge_ID_38, 2 of 102 at time 2024-06-03 15:21:15.431681
Dissolving for Merge_ID_67, 3 of 102 at time 2024-06-03 15:21:30.354332
Dissolving for Merge_ID_35, 4 of 102 at time 2024-06-03 15:22:02.655872
Dissolving for Merge_ID_83, 5 of 102 at time 2024-06-03 15:22:28.903382


In [199]:
#Permanent cell 12
#Export jpgs
if run_jpg:
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    project_path = aprx.filePath

    jpg_folder = model_output_folder + r'\jpg'
    if not os.path.isdir(jpg_folder): os.makedirs(jpg_folder) 


    # project_directory = os.path.dirname(project_path)

    layouts = aprx.listLayouts()
    export_fails = []

    for layout in layouts:

        if layout.mapSeries is not None:
            map_series = layout.mapSeries
            # Loop through all pages in the map series
            for page_number in range(1, map_series.pageCount + 1):
                map_series.currentPageNumber = page_number
                output_filename = os.path.join(jpg_folder, f"{map_series.pageRow.Drains_To}.jpg")
                try:
                    layout.exportToJPEG(output_filename, resolution=300)
                except:
                    print(f'WARNING! {map_series.pageRow.Drains_To} could not be made')
                    export_fails.append(map_series.pageRow.Drains_To)
                print (f'Printing jpg {page_number} of {map_series.pageCount} at time {datetime.datetime.now()}')

    print(f'the following pages failed: {export_fails}')            
    print("Export complete.")

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 12', b'Error', 0)
    raise ValueError("Error")



Printing jpg 1 of 458 at time 2024-06-03 13:41:53.490588
Printing jpg 2 of 458 at time 2024-06-03 13:41:57.274554
Printing jpg 3 of 458 at time 2024-06-03 13:42:00.614611
Printing jpg 4 of 458 at time 2024-06-03 13:42:04.145346
Printing jpg 5 of 458 at time 2024-06-03 13:42:07.265202
Printing jpg 6 of 458 at time 2024-06-03 13:42:10.769913
Printing jpg 7 of 458 at time 2024-06-03 13:42:14.453284
Printing jpg 8 of 458 at time 2024-06-03 13:42:17.646712
Printing jpg 9 of 458 at time 2024-06-03 13:42:20.645457
Printing jpg 10 of 458 at time 2024-06-03 13:42:23.862904
Printing jpg 11 of 458 at time 2024-06-03 13:42:26.707508
Printing jpg 12 of 458 at time 2024-06-03 13:42:29.270357
Printing jpg 13 of 458 at time 2024-06-03 13:42:32.897677
Printing jpg 14 of 458 at time 2024-06-03 13:42:36.051069
Printing jpg 15 of 458 at time 2024-06-03 13:42:39.481209
Printing jpg 16 of 458 at time 2024-06-03 13:42:42.583551
Printing jpg 17 of 458 at time 2024-06-03 13:42:45.645354
Printing jpg 18 of 458 

Printing jpg 141 of 458 at time 2024-06-03 13:49:21.931707
Printing jpg 142 of 458 at time 2024-06-03 13:49:25.143149
Printing jpg 143 of 458 at time 2024-06-03 13:49:27.940709
Printing jpg 144 of 458 at time 2024-06-03 13:49:30.554608
Printing jpg 145 of 458 at time 2024-06-03 13:49:33.712012
Printing jpg 146 of 458 at time 2024-06-03 13:49:36.679235
Printing jpg 147 of 458 at time 2024-06-03 13:49:39.774067
Printing jpg 148 of 458 at time 2024-06-03 13:49:42.864398
Printing jpg 149 of 458 at time 2024-06-03 13:49:45.366688
Printing jpg 150 of 458 at time 2024-06-03 13:49:48.626175
Printing jpg 151 of 458 at time 2024-06-03 13:49:51.022872
Printing jpg 152 of 458 at time 2024-06-03 13:49:53.836473
Printing jpg 153 of 458 at time 2024-06-03 13:49:56.160600
Printing jpg 154 of 458 at time 2024-06-03 13:49:59.337507
Printing jpg 155 of 458 at time 2024-06-03 13:50:02.179611
Printing jpg 156 of 458 at time 2024-06-03 13:50:05.354517
Printing jpg 157 of 458 at time 2024-06-03 13:50:08.5309

Printing jpg 279 of 458 at time 2024-06-03 13:56:25.590190
Printing jpg 280 of 458 at time 2024-06-03 13:56:28.369734
Printing jpg 281 of 458 at time 2024-06-03 13:56:33.059528
Printing jpg 282 of 458 at time 2024-06-03 13:56:40.069445
Printing jpg 283 of 458 at time 2024-06-03 13:56:42.434112
Printing jpg 284 of 458 at time 2024-06-03 13:56:45.338770
Printing jpg 285 of 458 at time 2024-06-03 13:56:48.778420
Printing jpg 286 of 458 at time 2024-06-03 13:56:52.138495
Printing jpg 287 of 458 at time 2024-06-03 13:56:54.646293
Printing jpg 288 of 458 at time 2024-06-03 13:56:57.431842
Printing jpg 289 of 458 at time 2024-06-03 13:57:00.340006
Printing jpg 290 of 458 at time 2024-06-03 13:57:03.182607
Printing jpg 291 of 458 at time 2024-06-03 13:57:06.153326
Printing jpg 292 of 458 at time 2024-06-03 13:57:08.933372
Printing jpg 293 of 458 at time 2024-06-03 13:57:11.839031
Printing jpg 294 of 458 at time 2024-06-03 13:57:14.996423
Printing jpg 295 of 458 at time 2024-06-03 13:57:17.3355

Printing jpg 417 of 458 at time 2024-06-03 14:03:58.873374
Printing jpg 418 of 458 at time 2024-06-03 14:04:01.156462
Printing jpg 419 of 458 at time 2024-06-03 14:04:09.686272
Printing jpg 420 of 458 at time 2024-06-03 14:04:12.342702
Printing jpg 421 of 458 at time 2024-06-03 14:04:17.093551
Printing jpg 422 of 458 at time 2024-06-03 14:04:22.113149
Printing jpg 423 of 458 at time 2024-06-03 14:04:25.716446
Printing jpg 424 of 458 at time 2024-06-03 14:04:28.309321
Printing jpg 425 of 458 at time 2024-06-03 14:04:32.404067
Printing jpg 426 of 458 at time 2024-06-03 14:04:34.966915
Printing jpg 427 of 458 at time 2024-06-03 14:04:37.422161
Printing jpg 428 of 458 at time 2024-06-03 14:04:39.904432
Printing jpg 429 of 458 at time 2024-06-03 14:04:42.816600
Printing jpg 430 of 458 at time 2024-06-03 14:04:45.278853
Printing jpg 431 of 458 at time 2024-06-03 14:04:47.841700
Printing jpg 432 of 458 at time 2024-06-03 14:04:50.375018
Printing jpg 433 of 458 at time 2024-06-03 14:04:53.0930

In [200]:
#Permanent cell 13
#Create spreadsheets

try:
    hex_blue = "ADD8E6"
    hex_yellow = "FFFACD"
    border_style = Side(style='thin', color='000000')
    border = Border(top=border_style, bottom=border_style, left=border_style, right=border_style)
    border_style_none = Side(style=None, color='000000')
    border_none = Border(top=border_style_none, bottom=border_style_none, left=border_style_none, right=border_style_none)

    excel_folder = model_output_folder + '\\Excel'
    img_folder = model_output_folder + '\\jpg'
    if not os.path.isdir(excel_folder): os.makedirs(excel_folder) 
    if not os.path.isdir(img_folder): os.makedirs(img_folder) 
    for id in node_df[('GENERAL INFO','ID')].unique():    
        node_single_df = node_df[node_df[('GENERAL INFO','ID')]==id].copy()
        node_single_df.reset_index(drop=True,inplace=True)

        id = id if not '/' in id else id.replace('/','_')
        muid = node_single_df.iloc[0,0]

        sheetpath = excel_folder + "\\" + id + ".xlsx"
        startrow = 13
        with pd.ExcelWriter(sheetpath) as writer:
            node_single_df.to_excel(writer, sheet_name=id,startrow=startrow)
            info_df.to_excel(writer, sheet_name=id,startrow=1,startcol=2)

            workbook = writer.book
            workbook.create_sheet("Map")

        workbook = load_workbook(sheetpath)    
        sheet1 = workbook[id]

        #Format infobox
        merged_range = sheet1.merged_cells
        for col in sheet1.iter_cols(min_col=3,max_col=5):
            for cell in col[1:2]:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")
            for cell in col[2:7]:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.fill = PatternFill(start_color=hex_yellow, end_color=hex_yellow, fill_type="solid")
                cell.border = border
        sheet1.column_dimensions['C'].width = 22 
        sheet1.column_dimensions['D'].width = 11 

        #Remove index
        for row in sheet1.iter_rows():
            for cell in row[:1]:
                cell.value = ''
                cell.border = border_none
        #Format main table header rows
        merged_range = sheet1.merged_cells
        for col in sheet1.iter_cols(min_col=2):
            for cell in col[startrow+1:startrow+2]:
                cell.alignment = Alignment(horizontal="center", vertical="center",wrap_text=True)
                cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")
            for cell in col[startrow:startrow+1]:
                if cell.coordinate in sheet1.merged_cells:
                    cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")        

        sheet1.column_dimensions['C'].width = 22 
        sheet1.column_dimensions['D'].width = 11   
        sheet1.column_dimensions['F'].width = 13
        sheet1.column_dimensions['H'].width = 13  
        sheet1.column_dimensions['V'].width = 13     

        sheet = workbook["Map"]

        # Add an image to the sheet
        img_path = img_folder + '\\' + muid + '.jpg'  # Replace with the path to your image
        img = Image(img_path)

        # Set the position for the image (e.g., 'B2' for cell B2)
        sheet.add_image(img, 'B2')

        workbook.save(sheetpath)


except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 13', b'Error', 0)
    raise ValueError("Error")



FileNotFoundError: [Errno 2] No such file or directory: '\\\\prdsynfile01\\lws_modelling\\SEWER_AREA_MODELS\\NSSA\\04_ANALYSIS_WORK\\RAWN_From_Model\\jpg\\6786.jpg'

In [ ]:
#Permanent cell 14
#Create HTMLs

try:
    if run_html:
        html_folder = model_output_folder + '\\HTML'
        
        shutil.copy2('style.css', html_folder + '\\style.css')
        shutil.copy2('script.js', html_folder + '\\script.js')

        for category in categories:
            area_type = category[0]
            area_names = category[1]
            header_start = category[2]

            f = open(html_folder + '\\Population_By_' + area_type + '_' + model_area + '.html', "w")
            f.write('<link rel="stylesheet" href="style.css">\n')
            f.write('<script src="script.js"></script>\n')
            f.write('<link rel="stylesheet" href="style.css">\n')
            f.write('<!DOCTYPE html>\n')
            f.write('<html>\n')
            f.write('<head>\n')
            f.write('<meta charset="utf-8">\n')
            f.write('</head>\n')
            f.write('<body>\n\n')

            f.write('<div class="tab">\n')
            for area_name in area_names:
                tab = area_name

            #     color = ps_dict[first_year]
            #     bg_color = color_dict[color][0]
            #     text_color = color_dict[color][1]

                f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
            f.write('</div>\n')

            pop_df = pop_dfss[0][2]

            for area_name in area_names:

                area_df = pop_df[pop_df[area_type]==area_name]
                area_df = area_df[['Year','Population']].groupby(['Year']).sum()

                f.write('<div id="' + area_name + '" class="tabcontent">\n') 
                f.write('<h1>' + area_name + '</h1>\n')

                f.write('<div class="sidenav">\n')

                f.write('<table style=\'width: 90%;\'>\n')
                f.write('<tr>\n')
                f.write('<th>Year</th>\n')
                f.write('<th>Population</th>\n')
                f.write('</tr>\n')

                for index, row in area_df.iterrows():
                    f.write('<tr>\n')
                    f.write('<td>'+ str(index) + '</td>\n')
                    population_with_separator = f"{int(row['Population']):,}"
                    f.write('<td>'+ population_with_separator + '</td>\n')

                    f.write('</tr>\n')
                f.write('</table>\n')

                for i in range(4):
                    f.write('<h1 style="color: white">End of tables</h1>\n')#Invisible, just to enable scroll to table bottoms

                f.write('</div>\n') #end sidenav


                f.write('<div class="main">\n')

                fig = go.Figure()


                fig.add_trace(go.Scatter(x=area_df.index, 
                                             y = area_df.Population, 
                                             mode='lines',name=pop_dfss[0][0],line=dict(width=5)))

                for pop_dfs in pop_dfss[1:]:
                    pop_df_past = pop_dfs[2]
                    area_df = pop_df_past[pop_df_past[area_type]==area_name]
                    area_df = area_df[['Year','Population']].groupby(['Year']).sum()
                    fig.add_trace(go.Scatter(x=area_df.index, 
                                             y = area_df.Population, 
                                             mode='lines',name=pop_dfs[0],line=dict(width=2)))

                fig.update_layout(
                    title=header_start + area_name,
                    autosize=False,
                    width = 1500,
                    height=850,
                    margin=dict(
                        l=50,
                        r=50,
                        b=50,
                        t=50,
                        pad=4
                        ),
                        yaxis_title = 'Population'
                    )

                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

                f.write('</div>\n') #end div main  

                f.write('</div>\n')  #end div tab   

                f.write('</body>\n')
            f.write('</html>\n')
            f.close()
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 14', b'Error', 0)
    raise ValueError("Error")



In [ ]:
#Obsolete cell
# #Dissolve catchments
# arcpy.env.addOutputsToMap = False
# if run_dissolve:
#     arcpy.management.CreateFileGDB(output_folder, gdb_name_dissolve)
#     dissolve_path = output_folder + '\\' + gdb_name_dissolve
#     arcpy.conversion.FeatureClassToFeatureClass('msm_Catchment', dissolve_path, 'msm_Catchment')
#     nodes = list(accumulation_df[('GENERAL INFO','NODE')].unique())
#     for i, node in enumerate(nodes):
#         print('Dissolving for node ' + str(i) + ' of ' + str(len(nodes)) + ' at time ' + str(datetime.datetime.now()))
#         catchment_df = accumulation_df[accumulation_df[('GENERAL INFO','NODE')]==node]
#         catchments = list(catchment_df[('GENERAL INFO','CATCHMENT')].unique())
#         arcpy.management.CalculateField(dissolve_path + '\\msm_Catchment', "Drains_To", "''", "PYTHON3")
#         with arcpy.da.UpdateCursor(dissolve_path + '\\msm_catchment', ['muid', 'Drains_To']) as cursor:
#             for row in cursor:
#                 if row[0] in catchments:
#                     row[1] = node
#                     cursor.updateRow(row)

#         query = "Drains_To = 'Test'"
#         arcpy.management.MakeFeatureLayer(dissolve_path + '\\msm_catchment', "temp_layer", "Drains_To = '" + node + "'")
#         dissolve_output = dissolve_path + '\\msm_Catchment_Dissolve_Single'
#         arcpy.management.Dissolve("temp_layer", dissolve_output, "Drains_To", "", "MULTI_PART")
#         arcpy.management.Delete("temp_layer")

#         arcpy.conversion.FeatureClassToFeatureClass(dissolve_path + '\\msm_Catchment_Dissolve_Single', dissolve_path, 'Node_Catchment_' + node)



In [ ]:
#Obsolete cell
#Append individual dissolved catchments to one layer.

# if run_dissolve_append:
#     nodes = list(accumulation_df[('GENERAL INFO','NODE')].unique())
#     for i, node in enumerate(nodes):    
#         print('Appending for node ' + str(i) + ' of ' + str(len(nodes)) + ' at time ' + str(datetime.datetime.now()))
#         if i == 0:
#             arcpy.conversion.FeatureClassToFeatureClass(dissolve_path + '\\Node_Catchment_' + node, out_path, 'Node_Catchment')
#         else:
#             arcpy.management.Append(dissolve_path + '\\Node_Catchment_' + node, "Node_Catchment", "NO_TEST")
